# Projet : planifier mes prochaines vacances ☀️

### Créons un script qui permet d'obtenir quelques informations sur tous les hôtels d'une ville donnée sur www.booking.com 🧙.

### Nous vous recommandons vivement d'utiliser Scrapy, ce sera beaucoup plus facile !

### Vous pouvez scrapper autant d'informations que vous le souhaitez, mais nous vous suggérons d'obtenir au moins :

## * Le nom de l'hôtel,
## * L'url de sa page booking.com,
## * Ses coordonnées : latitude et longitude,
## * La note donnée par les utilisateurs du site,
## * La description textuelle de l'hôtel.

### Ensuite, vous pouvez exécuter ce script pour plusieurs villes de la liste d'hier. Assurez-vous de sauvegarder les résultats dans des fichiers différents pour chaque ville et que le nom de la ville est stocké dans le nom du fichier (pour des raisons ultérieures 😉 ).

In [1]:
!pip install Scrapy

In [2]:
import os
import logging

import scrapy
from scrapy.crawler import CrawlerProcess

In [3]:
destination_name = 'Strasbourg'  # 'Chateau du Haut Koenigsbourg'

In [4]:
class Hotels(scrapy.Spider):
    # Name of your spider
    name = "hotels"

    # Starting URL
    start_urls = ['https://www.booking.com/index.fr.html']

    # Parse function for login
    def parse(self, response):
        # FormRequest used to login
        return scrapy.FormRequest.from_response(
            response,
            formdata={'ss': destination_name},
            callback=self.after_search
        )

    # Callback used after login
    def after_search(self, response):
        
        hotels = response.css('.sr_item')

        for h in hotels:
            yield {
                'name': h.css('.sr-hotel__name::text').get(),
                'url': "https://www.booking.com" + h.css('.hotel_name_link').attrib["href"],
                'coords': h.css('.sr_card_address_line a').attrib["data-coords"],
                'score': h.css('.bui-review-score__badge::text').get(),
                'description': h.css('.hotel_desc::text').get()
                
            }
        
        
        # Select the NEXT button and store it in next_page
        try:
            next_page = response.css('a.paging-next').attrib["href"]
        except KeyError:
            logging.info('No next page. Terminating crawling process.')
        else:
            yield response.follow(next_page, callback=self.after_search)

In [5]:
filename = "2_hotels_" + destination_name.replace(" ", "-") + ".json"

if filename in os.listdir('res/'):
    os.remove('res/' + filename)

process = CrawlerProcess(settings = {
    'USER_AGENT': 'Chrome/84.0 (compatible; MSIE 7.0; Windows NT 5.1)',
    'LOG_LEVEL': logging.INFO,
    "FEEDS": {
        'res/' + filename: {"format": "json"},
    }
})

process.crawl(Hotels)
process.start()

2022-02-25 21:17:54 [scrapy.utils.log] INFO: Scrapy 2.5.1 started (bot: scrapybot)
2022-02-25 21:17:54 [scrapy.utils.log] INFO: Versions: lxml 4.8.0.0, libxml2 2.9.12, cssselect 1.1.0, parsel 1.6.0, w3lib 1.22.0, Twisted 22.1.0, Python 3.9.7 | packaged by conda-forge | (default, Sep 29 2021, 19:20:46) - [GCC 9.4.0], pyOpenSSL 22.0.0 (OpenSSL 1.1.1l  24 Aug 2021), cryptography 36.0.1, Platform Linux-5.4.144+-x86_64-with-glibc2.31
2022-02-25 21:17:54 [scrapy.crawler] INFO: Overridden settings:
{'LOG_LEVEL': 20,
 'USER_AGENT': 'Chrome/84.0 (compatible; MSIE 7.0; Windows NT 5.1)'}
2022-02-25 21:17:54 [scrapy.extensions.telnet] INFO: Telnet Password: f01373de3bb949cd
2022-02-25 21:17:54 [scrapy.middleware] INFO: Enabled extensions:
['scrapy.extensions.corestats.CoreStats',
 'scrapy.extensions.telnet.TelnetConsole',
 'scrapy.extensions.memusage.MemoryUsage',
 'scrapy.extensions.feedexport.FeedExporter',
 'scrapy.extensions.logstats.LogStats']
2022-02-25 21:17:54 [scrapy.middleware] INFO: Ena